In [1]:
import pandas as pd
import networkx as nx
import random

from functions.sudoku import Sudoku

sociality is the number of rows (randomly chosen) that will come from the group best
memory is the number of rows (randomly chosen) that will come from the personal best
stability is the number of rows left over

In [29]:
class Swarm:
    def __init__(self, sudoku, pop_size = 40, num_neighbors = 4,
                sociality = 5 ,memory = 3, mutation_prob = 0.2, smart_mutation_prob = 0.2):
        import networkx as nx
        self.initial_sudoku = sudoku
        self.pop_size = pop_size
        self.num_neighbors = num_neighbors
        self.sociality = sociality
        self.memory = memory
        self.stability = 9 - sociality - memory
        self.mutation_prob = mutation_prob
        self.smart_mutation_prob = smart_mutation_prob
        self.graph = nx.random_regular_graph(num_neighbors, pop_size)
        self.population = [Sudoku(self.fill_sudoku_random(sudoku)) for _ in range(self.pop_size)]
        self.personal_bests = [x for x in self.population]
        self.group_best = [self.find_group_best(x) for x in range(pop_size)]
        
    def find_group_best(self, node):
        neighbor_sudokus = [self.population[x] for x in [node] + [_ for _ in self.graph[node]]]
        neighbor_costs = [x.cost for x in neighbor_sudokus]
        idx = neighbor_costs.index(min(neighbor_costs))
        return neighbor_sudokus[idx] 

    def fill_row_random(self, row):
        import random
        new_row = list()
        missing_values = set(range(1,10)).difference(set(row))
        rearranged = random.sample(missing_values, len(missing_values))
        for i in range(9):
            if row[i] == 0:
                new_row.append(rearranged.pop())
            else:
                new_row.append(row[i])
        return new_row
        
    def fill_sudoku_random(self, sudoku):
        import random       
        filled_sudoku = list()
        
        for row in sudoku.values:
            filled_sudoku.append(self.fill_row_random(row))
        return filled_sudoku
    
#     def mutate(self, sudoku):
#         import random
#         mutated_rows = random.sample(range(9), random.randint(1,9))
#         mutated = list()
#         for i in range(9):
#             if i in mutated_rows:
#                 mutated.append(self.fill_row_random(self.initial_sudoku.values[i]))
#             else:
#                 mutated.append(sudoku.values[i])
#         return Sudoku(mutated)
    
    def mutate(self, sudoku):
        import random
        sudoku_cost = sudoku.cost
        if sudoku_cost >= 12: 
            mutation_size = random.randint(1,9)
        elif sudoku_cost >= 8:
            mutation_size = 3
        elif sudoku_cost >= 4:
            mutation_size = 2
        elif sudoku_cost >= 1:
            mutation_size = 1
        else: 
            mutation_size = 0
        mutated_rows = random.sample(range(9), mutation_size)
        mutated = list()
        for i in range(9):
            if i in mutated_rows:
                mutated.append(self.fill_row_random(self.initial_sudoku.values[i]))
            else:
                mutated.append(sudoku.values[i])
        return Sudoku(mutated)
    
    def smart_mutate(self, sudoku):
        import random
        errors = sudoku.mark_errors()
        mutated = list()
        for i in range(9):
            new_row = list()
            counter = 0
            swapped = sorted(errors[i])[::-1]
            for j in range(9):
                if j in swapped:
                    new_row.append(sudoku.values[i][swapped[counter]])
                    counter += 1
                else:
                    new_row.append(sudoku.values[i][j])
            mutated.append(new_row)
        return Sudoku(mutated)

    def move(self, node):
        import random
        old_sudoku = self.population[node].values
        old_pb = self.personal_bests[node].values
        old_gb = self.group_best[node].values
        new_sudoku = [None] * 9
        scrambled = random.sample(range(9), 9)
        for i in scrambled[:self.sociality]:
            new_sudoku[i] = old_gb[i]
        for j in scrambled[self.sociality:self.sociality + self.memory]:
            new_sudoku[j] = old_pb[j]
        for k in scrambled[self.sociality + self.memory:]:
            new_sudoku[k] = old_sudoku[k]
        return Sudoku(new_sudoku)
        
    def make_new_population(self):
        mutation_prob = self.mutation_prob
        smart_mutation_prob = self.smart_mutation_prob
        import random
        import numpy as np
        new_pop = [self.move(_) for _ in range(self.pop_size)]
        mutations = random.sample(range(self.pop_size), int(mutation_prob * self.pop_size))
        for i in mutations:
            new_pop[i] = self.mutate(new_pop[i])
        smart_mutations = random.sample(range(self.pop_size), int(smart_mutation_prob * self.pop_size))
        for i in mutations:
            new_pop[i] = self.smart_mutate(new_pop[i])
        new_pb = [new_pop[i] if new_pop[i].cost < self.personal_bests[i].cost else self.personal_bests[i] for i in range(self.pop_size)]
        self.population = new_pop
        new_gb = [self.find_group_best(i) if self.find_group_best(i).cost < self.group_best[i].cost else self.group_best[i] for i in range(self.pop_size)]
        self.personal_bests = new_pb
        self.group_best = new_gb
        
    def change_neighbors(self):
        self.graph = nx.random_regular_graph(self.num_neighbors, self.pop_size)
        self.group_best = [self.find_group_best(x) for x in range(self.pop_size)]

In [30]:
easy = [[0,0,0,0,0,8,5,4,3],
       [0,2,5,6,1,0,7,0,9],
       [0,0,0,0,0,0,0,2,6],
       [0,3,0,0,0,2,0,6,7],
       [0,0,0,5,7,1,0,0,0],
       [7,8,0,4,0,0,0,1,0],
       [2,7,0,0,0,0,0,0,0],
       [1,0,9,0,4,3,2,5,0],
       [4,5,3,8,0,0,0,0,0]]

In [34]:
swarm = Swarm(Sudoku(easy), pop_size= 500, num_neighbors=20, mutation_prob=0.2, sociality=5, memory=3)

In [35]:
counter = 0
prev_best = min([x.cost for x in swarm.population])
#while (min([x.cost for x in swarm.population]) > 0):
while(prev_best > 0):
    swarm.make_new_population()
    print(min([x.cost for x in swarm.population]))
    new_best = min([x.cost for x in swarm.population])
    if prev_best == new_best:
        counter += 1
    else: counter = 0
    prev_best = new_best
    if counter > 40:
        print("Making new neighbors")
        swarm.change_neighbors()
        counter = 0

30
27
26
21
21
18
18
17
17
16
14
14
13
12
11
11
9
9
8
7
6
6
5
4
4
4
4
4
4
2
2
2
2
0


In [7]:
print(swarm.population[[x.cost for x in swarm.population].index(min([x.cost for x in swarm.population]))].values)

[[6, 9, 1, 7, 2, 8, 5, 4, 3], [3, 2, 5, 6, 1, 4, 7, 8, 9], [8, 4, 7, 9, 5, 3, 1, 2, 6], [5, 3, 4, 1, 8, 2, 9, 6, 7], [9, 6, 2, 5, 7, 1, 8, 3, 4], [7, 8, 3, 4, 6, 9, 2, 1, 5], [2, 7, 6, 8, 9, 5, 3, 4, 1], [1, 7, 9, 6, 4, 3, 2, 5, 8], [4, 5, 3, 8, 1, 7, 6, 9, 2]]


In [8]:
print(swarm.initial_sudoku.values)

[[0, 0, 0, 0, 0, 8, 5, 4, 3], [0, 2, 5, 6, 1, 0, 7, 0, 9], [0, 0, 0, 0, 0, 0, 0, 2, 6], [0, 3, 0, 0, 0, 2, 0, 6, 7], [0, 0, 0, 5, 7, 1, 0, 0, 0], [7, 8, 0, 4, 0, 0, 0, 1, 0], [2, 7, 0, 0, 0, 0, 0, 0, 0], [1, 0, 9, 0, 4, 3, 2, 5, 0], [4, 5, 3, 8, 0, 0, 0, 0, 0]]


In [9]:
[x.cost for x in swarm.population]

[13,
 16,
 13,
 13,
 46,
 13,
 13,
 13,
 13,
 43,
 13,
 21,
 16,
 13,
 13,
 13,
 13,
 13,
 26,
 21,
 13,
 13,
 13,
 13,
 13,
 13,
 53,
 27,
 16,
 13,
 20,
 13,
 13,
 20,
 13,
 13,
 21,
 37,
 44,
 13,
 42,
 13,
 13,
 13,
 17,
 13,
 14,
 13,
 13,
 13,
 13,
 13,
 41,
 45,
 13,
 36,
 13,
 24,
 19,
 13,
 13,
 13,
 13,
 27,
 13,
 13,
 13,
 22,
 13,
 13,
 13,
 24,
 30,
 13,
 13,
 13,
 35,
 17,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 24,
 23,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 13,
 13,
 13,
 13,
 13,
 24,
 13,
 13,
 13,
 13,
 13,
 18,
 13,
 30,
 13,
 17,
 13,
 13,
 13,
 18,
 13,
 17,
 13,
 24,
 24,
 13,
 13,
 13,
 19,
 22,
 13,
 13,
 13,
 28,
 13,
 13,
 13,
 26,
 20,
 43,
 13,
 13,
 13,
 13,
 13,
 17,
 13,
 13,
 33,
 13,
 17,
 13,
 13,
 49,
 34,
 13,
 13,
 40,
 36,
 13,
 43,
 13,
 13,
 13,
 13,
 21,
 13,
 13,
 22,
 13,
 13,
 13,
 13,
 25,
 13,
 13,
 29,
 13,
 13,
 13,
 13,
 13,
 13,
 36,
 44,
 13,
 13,
 13,
 14,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 21,
 42,
 13,
 16,
 28,
 13,


In [38]:
print(swarm.population[9].values)

[[6, 1, 7, 2, 9, 8, 5, 4, 3], [3, 2, 5, 6, 1, 4, 7, 8, 9], [8, 4, 9, 3, 5, 7, 1, 2, 6], [5, 3, 1, 9, 8, 2, 4, 6, 7], [4, 9, 6, 5, 7, 1, 8, 3, 2], [7, 8, 2, 4, 3, 6, 9, 1, 5], [2, 7, 8, 1, 6, 5, 3, 9, 4], [1, 6, 9, 7, 4, 3, 2, 5, 8], [4, 5, 3, 8, 2, 9, 6, 7, 1]]


In [59]:
class Sudoku:
    def __init__(self, values):
        self.values = values
        self.cost = self.cost(values)
    
    def count_zeros(self, sudoku):
        count = 0
        for i in range(9): count += sudoku[i].count(0)
        return count
    
    def transpose(self, sudoku):
        trans_sudoku = [[0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0],
                        [0,0,0,0,0,0,0,0,0]]
        for i in range(9):
            for j in range(9):
                trans_sudoku[i][j] = sudoku[j][i]
        return(trans_sudoku)
        
    def cost(self,sudoku):
        penalty = 0
        for i in range(9):
            penalty += 9-len(set(sudoku[i]))
            trans_sudoku = self.transpose(sudoku)
        for i in range(9):
            penalty += 9 - len(set(trans_sudoku[i]))
        for i in range(3):
            for j in range(3):
                values = set()
                for a in range(3):
                    for b in range(3):
                        values.add(sudoku[3*i+a][3*j+b])
                penalty += 9 - len(values)
        return(penalty)
    
    def mark_errors(self):
        # return 9 lists, one for each row - each list contains those indices there either there is a duplicate in the columns or blocks
        errors = []
        for i in range(9):
            row_errors = []
            for j in range(9):
                if [x[j] for x in self.values].count(self.values[i][j]) > 1:
                    row_errors.append(j)
                block_values = self.values[3*int(i/3)][3*int(j/3):3*(int(j/3) + 1)] + \
                self.values[3*int(i/3) + 1][3*int(j/3):3*(int(j/3) + 1)] + \
                self.values[3*int(i/3) + 2][3*int(j/3):3*(int(j/3) + 1)]
                if block_values.count(self.values[i][j]) > 1:
                    row_errors.append(j)
            errors.append(list(set(row_errors)))
        return errors

In [10]:
sudoku = Sudoku(swarm.population[[x.cost for x in swarm.population].index(min([x.cost for x in swarm.population]))].values)

In [11]:
sudoku.cost

13

In [12]:
sudoku.mark_errors()

[[7], [3, 4], [5], [1, 3], [5], [2, 6], [1, 3, 7], [1, 3, 5, 6], [8, 2, 3, 4]]

In [17]:
sudoku.values

[[6, 9, 1, 7, 2, 8, 5, 4, 3],
 [3, 2, 5, 6, 1, 4, 7, 8, 9],
 [8, 4, 7, 9, 5, 3, 1, 2, 6],
 [5, 3, 4, 1, 8, 2, 9, 6, 7],
 [9, 6, 2, 5, 7, 1, 8, 3, 4],
 [7, 8, 3, 4, 6, 9, 2, 1, 5],
 [2, 7, 6, 8, 9, 5, 3, 4, 1],
 [1, 7, 9, 6, 4, 3, 2, 5, 8],
 [4, 5, 3, 8, 1, 7, 6, 9, 2]]

In [16]:
swarm.smart_mutate(sudoku).values

[[6, 9, 1, 7, 2, 8, 5, 4, 3],
 [3, 2, 5, 1, 6, 4, 7, 8, 9],
 [8, 4, 7, 9, 5, 3, 1, 2, 6],
 [5, 1, 4, 3, 8, 2, 9, 6, 7],
 [9, 6, 2, 5, 7, 1, 8, 3, 4],
 [7, 8, 2, 4, 6, 9, 3, 1, 5],
 [2, 4, 6, 8, 9, 5, 3, 7, 1],
 [1, 2, 9, 3, 4, 6, 7, 5, 8],
 [4, 5, 2, 1, 8, 7, 6, 9, 3]]